# Predict parent neuron
Using Jasper's idea to predict parent neurons for nuclei, Thank you so much, Jasper.

In [1]:
# libraries 1
import numpy as np
import pyperclip
import pandas as pd
import pyperclip
from cloudvolume import CloudVolume, view, Bbox
from nglui import statebuilder,annotation,easyviewer,parser
from nglui.statebuilder import *
import json
from tqdm import tqdm
import argparse

from PIL import Image
import matplotlib.pyplot as plt
import sys
import os
import csv

from tifffile.tifffile import imwrite

sys.path.append(os.path.abspath("../segmentation"))
import authentication_utils as auth
import rootID_lookup as IDlook
sys.path.append(os.path.abspath("../synapses"))
import connectivity_utils

In [3]:
np.random.seed(123)
# read csv file
df = pd.read_csv('../Output/body_info_Aug2021.csv', header=0)
len(df)

17076

# A. Create Neuroglancer links
based on https://github.com/bjm5164/rotation_projects/blob/main/statebuilder_examples.ipynb

In [ ]:
imgTokyo = ImageLayerConfig(name = 'FANC_EM_Tokyo',
                                    source = auth.get_cv_path('Image_Tokyo')['url'])
img = ImageLayerConfig(name = 'FANC_EM',
                                    source = auth.get_cv_path('Image')['url'])
nuc = ImageLayerConfig(name = 'nuclei_Jul2021',
                                    source = auth.get_cv_path('nuclei_map_Jul2021')['url'])      
nuc_seg = ImageLayerConfig(name = 'nuclei_Jul2021_seg',
                                    source = auth.get_cv_path('nuclei_seg_Jul2021')['url'])      
# volume = SegmentationLayerConfig(name = 'volume_outlines',
#                                    source = auth.get_cv_path('Image')['url'])   

LINK=[]
k=100
minidfs = [df.loc[i:i+k-1, :] for i in range(0, len(df), k)]
for dftemp in minidfs:
    seg = SegmentationLayerConfig(name = 'FANC_production_segmentation',
                                        active = True,
                                        source = auth.get_cv_path('FANC_production_segmentation')['url'],
                                        selected_ids_column='nuc_segID')                                 
    sb = StateBuilder(layers=[imgTokyo, seg, img, nuc, nuc_seg],resolution=[4.3,4.3,45]) # view_kws=view_options
    output = sb.render_state(dftemp) #return_as='html'
    LINK.append(output)

In [ ]:
# https://api.zetta.ai/json/post

In [ ]:
# save into csv
LINK2 = pd.DataFrame(LINK)
LINK2.to_csv('../Output/nuc_links.csv', index=False, header=False)

# B. Test my soma list with other table

In [ ]:
# T1MN_somas test
MNsomas = pd.read_csv('../Output/T1MN_somas.csv', header=0)
MNsomas['pt_root_id'].values

exist = []
no_exist = []
for i in MNsomas['pt_root_id'].values:
    if i in df['body_segID'].values:
        exist.append(i)
    else:
        no_exist.append(i)

print(no_exist)

In [ ]:
check = 648518346486520985
print (MNsomas.loc[MNsomas['pt_root_id'] == check])

In [ ]:
nuc = 648518346481315476
df[df['nuc_segID']==nuc]

# C. Summarize count data

In [ ]:
count = pd.read_csv('../Output/ncount_merged.csv', header=None)
count2 = count.T.dropna()
# len(count2) 
# 15094
count3 = count2.sort_values(by=0)
count4 = count3.astype('int')
count4.columns = ['index', 'count']

In [ ]:
sv_size_thres=4000
chosen = count4.loc[count4['count'] <= sv_size_thres]
chosen['index'].values

In [ ]:
test = df.loc[chosen['index'].values]
test[0:999]
rantest = test.iloc[np.random.choice(len(test), round(len(test)/5), replace=False)]

In [ ]:
# nuclei itself
imgTokyo = ImageLayerConfig(name = 'FANC_EM_Tokyo',
                                    source = auth.get_cv_path('Image_Tokyo')['url'])
img = ImageLayerConfig(name = 'FANC_EM',
                                    source = auth.get_cv_path('Image')['url'])
nuc = ImageLayerConfig(name = 'nuclei(old)',
                                    source = auth.get_cv_path('nuclei_map')['url'])                                    
# volume = SegmentationLayerConfig(name = 'volume_outlines',
#                                    source = auth.get_cv_path('Image')['url'])   

seg = SegmentationLayerConfig(name = 'FANC_production_segmentation',
                                        active = True,
                                        source = auth.get_cv_path('FANC_production_segmentation')['url'],
                                        selected_ids_column='segIDs')                                 
sb = StateBuilder(layers=[imgTokyo, seg, nuc, img],resolution=[4.3,4.3,45]) # view_kws=view_options
output = sb.render_state(rantest, return_as='html')
output

In [ ]:
sv_size_thres=4000
notchosen = count4.loc[count4['count'] > sv_size_thres+2000]
test2 = df.loc[notchosen['index'].values]
test2

In [ ]:
rantest2 = test2.iloc[np.random.choice(len(test2), round(len(test2)/5), replace=False)]
output = sb.render_state(rantest2, return_as='html')
output

# D. Get premotor inputs

In [3]:
# get premotor inputs
MN = pd.read_csv('../Output/MNs.csv', header=0)
MN

,Unnamed: 0,name,cell_type,pt_position,pt_root_id,soma_x_nm,soma_y_nm,soma_z_nm,found,table_name
0,0,MN_A101_T1L,NaN,"[13329, 114635, 1861]",648518346495822182,57314.7,492930.5,83745.0,NaN,T1MN_somas
1,1,MN_A101_T1R,NaN,"[63394, 114393, 2304]",648518346494371895,272594.2,491889.9,103680.0,NaN,T1MN_somas
2,2,MN_A102_T1L,NaN,"[11729, 116735, 1478]",648518346495234895,50434.7,501960.5,66510.0,NaN,T1MN_somas
3,3,MN_A102_T1R,NaN,"[63084, 116459, 2863]",648518346513703705,271261.2,500773.7,128835.0,NaN,T1MN_somas
4,4,MN_A103_T1R,NaN,"[59659, 115855, 2065]",648518346482874378,256533.7,498176.5,92925.0,NaN,T1MN_somas
...,...,...,...,...,...,...,...,...,...,...
117,117,MN_V401_T1L,NaN,"[17144, 89099, 2005]",648518346509893335,73719.2,383125.7,90225.0,NaN,T1MN_somas
118,118,MN_V402_T1L,NaN,"[23167, 87333, 2184]",648518346481306305,99618.1,375531.9,98280.0,NaN,T1MN_somas
119,119,MN_V501_T1R,NaN,"[54883, 84397, 2319]",648518346497794202,235996.9,362907.1,104355.0,NaN,T1MN_somas
120,120,MN_V601_T1L,NaN,"[21122, 88740, 2060]",648518346488475633,90824.6,381582.0,92700.0,NaN,T1MN_somas


# E. point map

In [5]:
# color_column not supported for point annotation
imgTokyo = ImageLayerConfig(name = 'FANC_EM_Tokyo',
                                    source = auth.get_cv_path('Image_Tokyo')['url'])
img = ImageLayerConfig(name = 'FANC_EM',
                                    source = auth.get_cv_path('Image')['url'])
nuc = ImageLayerConfig(name = 'nuclei_Jul2021',
                                    source = auth.get_cv_path('nuclei_map_Aug2021')['url'])      
nuc_seg = ImageLayerConfig(name = 'nuclei_Jul2021_seg',
                                    source = auth.get_cv_path('nuclei_seg_Aug2021')['url'])      
# volume = SegmentationLayerConfig(name = 'volume_outlines',
#                                    source = auth.get_cv_path('Image')['url'])   

seg = SegmentationLayerConfig(name = 'FANC_production_segmentation',
                                    active = True,
                                    source = auth.get_cv_path('FANC_production_segmentation')['url'])

df2 = df.assign(pt_position=[*zip(df.x, df.y, df.z)])
points = PointMapper(point_column='pt_position')
ann = AnnotationLayerConfig(name='nuc_center', mapping_rules=points)      
sb = StateBuilder(layers=[imgTokyo, seg, img, nuc, nuc_seg, ann],resolution=[4.3,4.3,45]) # view_kws=view_options

client,token = auth.get_client()

state = json.loads(sb.render_state(df2, return_as='json'))

out =auth.get_cv_path('neuroglancer_base')['url'] + '?json_url={path}{state_id}'.format(path=auth.get_cv_path('json_server')['url'],state_id=client.state.upload_state_json(state))
out
# https://neuromancer-seung-import.appspot.com/?json_url=https://api.zetta.ai/json/692331708447824330
# https://github.com/htem/FANC_auto_recon/blob/main/proofreading/proofreading_utils.py

'https://neuromancer-seung-import.appspot.com/?json_url=https://api.zetta.ai/json/784225246198071353'